In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
sft_model_path =  "ckpts/sft_lama_GPT2_commit_lr5e-7/checkpoint_10000/hf_model"
ppo_model_path = "ckpts/ppo_lama_GPT2_commit30_hedge0_0_idk10_lr5e-6/checkpoint_10000/hf_model"

In [14]:
sft_generation_categories = np.load(sft_model_path+"/generation_categories.npy")
ppo_generation_categories = np.load(ppo_model_path+"/generation_categories.npy")
# np.stack([np.array(kwargs["split"]), commit_correct, commit_wrong, dont_know, wrong, hedge_correct, hedge_wrong], axis=1)

In [44]:
ood_idxs = np.where(sft_generation_categories[:, 0]==3)[0]
test_idxs = np.where(sft_generation_categories[:, 0]==2)[0]

In [27]:
sft_correct  = np.where(sft_generation_categories[:, 1]==1)[0]
sft_correct_ppo_correct = len(np.where(ppo_generation_categories[sft_correct, 1]==1)[0])
sft_correct_ppo_wrong = len(np.where(ppo_generation_categories[sft_correct, 2]==1)[0])
sft_correct_ppo_idk = len(np.where(ppo_generation_categories[sft_correct, 3]==1)[0])
print(np.array([sft_correct_ppo_correct,sft_correct_ppo_wrong,sft_correct_ppo_idk])/(sft_correct_ppo_correct+sft_correct_ppo_wrong+sft_correct_ppo_idk))

[0.62535211 0.04741784 0.32723005]


In [29]:
sft_wrong  = np.where(sft_generation_categories[:, 2]==1)[0]
sft_wrong_ppo_correct = len(np.where(ppo_generation_categories[sft_wrong, 1]==1)[0])
sft_wrong_ppo_wrong = len(np.where(ppo_generation_categories[sft_wrong, 2]==1)[0])
sft_wrong_ppo_idk = len(np.where(ppo_generation_categories[sft_wrong, 3]==1)[0])
print(np.array([sft_wrong_ppo_correct,sft_wrong_ppo_wrong,sft_wrong_ppo_idk])/(sft_wrong_ppo_correct+sft_wrong_ppo_wrong+sft_wrong_ppo_idk))

[0.05189137 0.30407371 0.64403492]


In [134]:
answer_log_probs_mean = np.load(sft_model_path+"/answer_log_probs_mean.npy")
answer_log_probs_sum = np.load(sft_model_path+"/answer_log_probs_sum.npy")


In [30]:
sft_hidden_states = np.load(sft_model_path+"/hidden_states.npy")
ppo_hidden_states = np.load(ppo_model_path+"/hidden_states.npy")

In [111]:
# np.linalg.norm(sft_hidden_states[1][sft_correct]-sft_hidden_states[1].mean(axis=0), axis=-1).std()
# np.linalg.norm(sft_hidden_states[1][sft_wrong]-sft_hidden_states[1].mean(axis=0), axis=-1).std()

ppo_idk_idxs = np.where(ppo_generation_categories[:, 3] == 1)[0]
ppo_not_idk_idxs = np.where(ppo_generation_categories[:, 3] == 0)[0]

# plt.hist(np.linalg.norm(sft_hidden_states[1][sft_correct]-sft_hidden_states[1].mean(axis=0), axis=-1), density=True, alpha=0.5)
# plt.hist(np.linalg.norm(sft_hidden_states[1][sft_wrong]-sft_hidden_states[1].mean(axis=0), axis=-1), density=True, alpha=0.5)
# plt.savefig("fig.png")


In [154]:
from sklearn.linear_model import SGDRegressor

for layer in range(13):
    X = sft_hidden_states[layer][test_idxs]
    # y = sft_generation_categories[:, 1][test_idxs]
    y = np.e**answer_log_probs_sum[test_idxs]
    reg = SGDRegressor().fit(X[:2500], y[:2500])
    print(layer)
    print(reg.score(X[:2500], y[:2500]))
    print(reg.score(X[2500:], y[2500:]))
    # print(reg.score(sft_hidden_states[layer][ood_idxs], sft_generation_categories[:, 1][ood_idxs]))
    print(reg.score(sft_hidden_states[layer][ood_idxs], np.e**answer_log_probs_sum[ood_idxs]))

0
0.2634764415985984
0.2192002868339371
-0.0017637407285688855
1
-7.100216281439427e+35
-3.013837937438014e+35
-1.296957962930272e+33
2
-2.43158396625009e+36
-1.1490348496094731e+36
-5.227925904302104e+33
3
-5.919449109072684e+36
-3.051664696601109e+36
-2.3834493380417064e+34
4
-8.158362715295396e+38
-2.941605883592354e+38
-7.53633446589719e+35
5
-3.7172180063781644e+40
-1.3010313804921398e+40
-2.2803052080495825e+37
6
-5.676365857663217e+40
-2.0511434610126204e+40
-5.205841633038986e+37
7
-2.0431731435426294e+41
-7.173248096659118e+40
-1.850272943996596e+38
8
-3.6612308570349885e+41
-1.2370113100687811e+41
-5.058327873698531e+38
9
-2.399958077636137e+41
-8.624925207365362e+40
-1.994887503968709e+38
10
-1.1287821119562472e+42
-3.690972016297945e+41
-4.161261421270609e+38
11
-1.4217442188402426e+43
-4.807945032668427e+42
-7.394748098241079e+39
12
-1.561750682191945e+43
-5.5051233762798715e+42
-8.793973872462887e+39


In [150]:
from sklearn.linear_model import LogisticRegression

for layer in range(13):
    X = sft_hidden_states[layer][test_idxs]
    # y = sft_generation_categories[:, 1][test_idxs]
    y = ppo_generation_categories[:, 3][test_idxs]
    reg = LogisticRegression(C=.1).fit(X[:2500], y[:2500])
    print(layer)
    print(reg.score(X[:2500], y[:2500]))
    print(reg.score(X[2500:], y[2500:]))
    # print(reg.score(sft_hidden_states[layer][ood_idxs], sft_generation_categories[:, 1][ood_idxs]))
    print(reg.score(sft_hidden_states[layer][ood_idxs], ppo_generation_categories[:, 3][ood_idxs]))

0
0.5888
0.6213183730715287
0.47264985260399606
1
0.8784
0.8653576437587658
0.7107762856207009
2
0.8316
0.8176718092566619
0.6528005240746806
3
0.91
0.9158485273492286
0.6917785784474287
4
0.9264
0.9396914446002805
0.7179823124795284
5
0.9328
0.9396914446002805
0.7061906321650835
6
0.9356
0.94109396914446
0.7160170324271209
7
0.9472
0.9453015427769986
0.7183098591549296
8
0.9532
0.9424964936886395
0.7029151654110711
9
0.9576
0.9453015427769986
0.6862102849656077
10
0.9652
0.9438990182328191
0.6940714051752375
11
0.9312
0.9186535764375876
0.6737635113003603
12
0.9204
0.9256661991584852
0.6796593514575827


In [133]:
layer = 7
X = sft_hidden_states[layer][test_idxs]
# y = sft_generation_categories[:, 1][test_idxs]
y = ppo_generation_categories[:, 3][test_idxs]
reg = LogisticRegression(C=1).fit(X[:2500], y[:2500])
print(layer)
print(reg.score(X[:2500], y[:2500]))
print(reg.score(X[2500:], y[2500:]))
# print(reg.score(sft_hidden_states[layer][ood_idxs], sft_generation_categories[:, 1][ood_idxs]))
print(reg.score(sft_hidden_states[layer][ood_idxs], ppo_generation_categories[:, 3][ood_idxs]))

7
0.9724
0.938288920056101
0.7834916475597773


/data/katie_kang/miniconda3/envs/trlx/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [124]:
(np.dot(X, reg.coef_[0]))

array([-0.86624331,  5.35596557, -0.50840423, ..., -0.93292231,
        2.79925149,  5.17442338])

In [121]:
reg.coef_.shape

(1, 768)

In [117]:
reg.intercept_

array([-0.00555812])

In [66]:
1-sft_generation_categories[:, 1][ood_idxs].sum()/len(sft_generation_categories[:, 1][ood_idxs])

0.6056338028169015

In [68]:
1-y.sum()/len(y)

0.7117958294428883

In [114]:
1 - ppo_generation_categories[:, 3][ood_idxs].sum()/len(ppo_generation_categories[:, 3][ood_idxs])

0.5273501473960039